In [1]:
from collections import Counter
import numpy as np
import time

In [2]:

NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [4]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

In [5]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
play(board, 5, 1)
print(board)
print(eval_board(board, 1))
eval_board(board, -1)

[[-1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
0.86


0.8

In [6]:
print(board)
board.all()

[[-1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


False

In [23]:
class connect4:
    def __init__(self):
        self.initialize_game()

    def initialize_game(self):
        self.board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
        play(self.board, 0, 1)
        play(self.board, 0, -1)
        play(self.board, 0, 1)
        play(self.board, 0, -1)
        play(self.board, 0, 1)
        play(self.board, 0, -1)
        play(self.board, 1, 1)
        play(self.board, 1, -1)
        play(self.board, 1, 1)
        play(self.board, 1, -1)
        play(self.board, 1, 1)
        play(self.board, 1, -1)
        play(self.board, 5, 1)
        play(self.board, 5, -1)
        play(self.board, 5, 1)
        play(self.board, 5, -1)
        play(self.board, 5, 1)
        play(self.board, 5, -1)
        play(self.board, 6, 1)
        play(self.board, 6, -1)
        play(self.board, 6, 1)
        play(self.board, 6, -1)
        play(self.board, 6, 1)
        play(self.board, 6, -1)
        self.player_turn = 1
    
    def check_end(self):
        if four_in_a_row(self.board, 1):
            # Alice won
            return 1
        elif four_in_a_row(self.board, -1):
            # Bob won
            return -1
        if self.board.all():
            #IT IS A TIE
            return 0
        return None

    def max(self):
        result = self.check_end()
        if result == 1:
            return (1, 0)
        elif result == -1:
            return (-1, 0)
        elif result == 0:
            return (0, 0)
        max_value = -2
        max_idx = None
        for i in valid_moves(self.board):
            play(self.board, i, 1)
            (value, min_idx) = self.min()
            if value > max_value:
                max_value = value
                max_idx = i
            take_back(self.board, i)
        return (max_value, max_idx)
            
    
    def min(self):
        result = self.check_end()
        if result == 1:
            return (1, 0)
        elif result == -1:
            return (-1, 0)
        elif result == 0:
            return (0, 0)
        min_value = 2
        min_idx = None
        for i in valid_moves(self.board):
            play(self.board, i, 1)
            (value, max_idx) = self.max()
            if value < min_value:
                min_value = value
                min_idx = i
            take_back(self.board, i)
        return (min_value, min_idx)
    
    def play(self):
        while True:
            print(self.board)
            self.result = self.check_end()
            if self.result != None:
                if self.result == 1:
                    print('The winner is Player 1')
                elif self.result == -1:
                    print('The winner is Player -1')
                elif self.result == 0:
                    print("It's a tie!")
                self.initialize_game()
                return
            if self.player_turn == -1:
                while True:
                    start = time.time()
                    (m, qx) = self.min()
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Recommended move: X = {}'.format(qx))
                    idx = int(input('Insert the column '))
                    qx = idx

                    if idx in valid_moves(self.board):
                        play(self.board, idx, -1)
                        self.player_turn = 1
                        break
                    else:
                        print('The move is not valid! Try again.')

            # If it's AI's turn
            else:
                (m, index) = self.max()
                play(self.board, index, 1)
                self.player_turn = -1


In [24]:
g = connect4()
g.play()

[[ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]]
[[ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]]
Evaluation time: 0.0442986s
Recommended move: X = 2
[[ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]
 [ 1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]]
[[ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]
 [ 1  1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]]
Evaluation time: 0.097404s
Recommended move: X = 2
[[ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]
 [ 1  1 -1  0  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]]
[[ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]
 [ 1  1 -1  1  0  0]
 [-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1 -1]]
Evaluatio